
No último handout conseguimos treinar algumas redes neurais (tradicionais e convolucionais) e obtivemos avaliações muito boas em nosso conjunto de treinamento. Este cenário, porém, não é realista: as imagens estão perfeitamente registradas. 

**Conceito**: em um pipeline de visão computacional a etapa de *registro* consiste em alinhas imagens de uma mesma cena (ou objeto) para que elas mostrem nas mesmas regiões da imagem os mesmos conteúdos. 

![](nao_registrada.png)

As duas imagens acima não estão registradas pois os números 4 não estão alinhados. Isto pode causar uma série de problemas em algoritmos de classificação de imagens. Vamos iniciar esta aula quantificando estes problemas.

**Exercício**: cole abaixo sua melhor rede neural da aula anterior. Inclua tudo necessário para que o modelo seja criado e treinado e para que os dados de treinamento sejam baixados. 


In [0]:
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
%matplotlib inline

(X, y), (Xt, yt) = mnist.load_data()
print('# Imagens:', y.shape[0])
print('Tamanho:', X[0].shape)
print('Primeiro número:', y[0])
plt.imshow(X[0], cmap='gray')

In [0]:
import numpy as np

Xlr = np.reshape(X, (60000, 784))
Xtlr = np.reshape(Xt, (10000, 784))

In [0]:
from keras.utils import to_categorical

y_oe = to_categorical(y, 10)
yt_oe = to_categorical(yt, 10)

In [0]:
Xnn = Xlr/255
Xtnn = Xtlr/255

In [0]:
Xcnn = Xnn.reshape(Xnn.shape[0], 28, 28, 1).astype('float32')
Xtcnn = Xtnn.reshape(Xtnn.shape[0], 28, 28, 1).astype('float32')

In [0]:
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential


model_cnn = Sequential()

model_cnn.add(layers.Conv2D(32, (5, 5), input_shape=(28, 28, 1), activation='relu'))
model_cnn.add(layers.MaxPooling2D(pool_size=(2, 2)))
model_cnn.add(layers.Flatten())
model_cnn.add(layers.Dense(128, activation='relu'))
model_cnn.add(layers.Dense(10, activation='softmax'))
model_cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
model_cnn.fit(Xcnn, y_oe, validation_data=(Xtcnn, yt_oe), batch_size=200, verbose=2, epochs = 10)

Agora que você já tem uma rede neural para testarmos, vamos explorar a classe `tf.keras.preprocessing.image.ImageDataGenerator`. Pesquise o quê ela faz e escreva abaixo um resumo.

Essa classe performa as transformações de translação e rotação nas imagens "on demand", sem guardá-las na memória.

**Exercício**: instancie um `ImageDataGenerator` que permita deslocamentos horizontais e verticais de até 5 pixels, rotações de até 25 graus e zoom de até 25% do tamanho da imagem. Gere, então, 10 variações de uma imagem a sua escolha de `Xt` e mostre-as usando matplotlib. 

**Dicas**:

* Todas as informações estão presentes na documentação desta classe. 
* Procure pelo método `random_transform` para transformar uma imagem 
* Antes de passar as imagens para a rede convolucional não se esqueça de deixar todos seus elementos no intervalo $[0, 1]$.
* Não se esqueça de usar `reshape` sempre que os dados forem pedidos em um formato diferente de entrada. 

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

imageData = ImageDataGenerator(width_shift_range=5, height_shift_range=5, rotation_range=25, zoom_range=0.25)

lista_random = []

for i in range(10):
  new_random = imageData.random_transform(Xtnn[0].reshape((28,28,1)))
  lista_random.append(new_random)
  print(np.argmax(model_cnn.predict(new_random.reshape((1,28,28,1)))))
  plt.imshow(new_random.reshape((28,28)), cmap='gray')
  plt.show()


**Exercício**: aplique seu classificador a estas imagens e verifique sua acurácia.

**Dicas**:

* novamente, `reshape` é seu amigo
* o método `predict` retorna probabilidades, use `np.argmax` para encontrar o índice do maior valor.

Já feito na célula acima

Você deve ter observado que em vários exemplos a resposta do seu classificador foi incorreta. O erro obtido foi muito maior que o mostrado na avaliação da aula anterior! Isto ocorre pois nosso classificador foi treinado com um conjunto de dados com uma distribuição diferente Ou seja, se ele nunca viu um 4 deslocado para a direita vai supor que isto é impossível. 

Uma estratégia comumente utilizada é incluir, no treinamento, variações pequenas nas imagens exatamente como geramos nas células acima. Desta maneira pode-se conseguir classificadores que sejam menos sensíveis as posições dos dígitos nas imagens (mais genericamente, na posição dos objetos dentro de uma cena). 

Dá-se o nome de **Data Augmentation** para qualquer procedimento que siga este fluxo de "turbinar" o conjunto de treinamento com variações de suas imagens de modo a deixar o classificador mais robusto. Naturalmente, ele será robusto aos tipos de transformações inclusas: um classificador não se tornará magicamente robusto a rotações se elas não estiverem presentes no conjunto de treinamento.

O principal uso de `ImageDataGenerator` é criar uma lista infinita de imagens escolhidas aleatoriamente e transformá-las também de maneira aleatória. Isto nos permite incluir no treinamento todas aquelas variações vistas nos exercícios anteriores. 

**Exercício**: use sua instância de `ImageDataGenerator` para treinar novamente seu modelo, mas desta vez incluindo as variações que mostramos anteriormente. 

**Dicas**:

* sempre consulte a documentação das classes que usamos
* pesquisar por *Python Generators* ajuda a desfazer a mágica por trás do funcionamento de `ImageDataGenerator`, mas não é obrigatório
* o método `.flow(X, y)` é seu amigo

In [0]:
model_cnn.fit(imageData.flow(Xcnn, y_oe), epochs = 10)

**Exercício**: agora que temos um classificador mais robusto, faça novamente o teste com as imagens deslocadas. Os resultados melhoraram?

In [0]:
for i in range(10):
  new_random = imageData.random_transform(Xtnn[10].reshape((28,28,1)))
  print(np.argmax(model_cnn.predict(new_random.reshape((1,28,28,1)))))
  plt.imshow(new_random.reshape((28,28)), cmap='gray')
  plt.show()

# Parte final

**Exercício**: tire uma foto de 5 números escritos à mão por você. Separe cada um dos números, redimensione-os para uma imagem $28\times 28$, passe cada um pelo seu melhor modelo desta aula e retorne a leitura do número por inteiro.

**Dica**: sua foto pode conter somente a folha em branco e os dígitos em preto. Você pode supor que nenhuma parte de um dígito encosta no outro e que a imagem é binarizável usando uma operação simples de limiar (*threshold*).

In [0]:
import cv2 as cv
img_zero = cv.imread("zero.jpg", cv.IMREAD_GRAYSCALE)
img_zero = (255-img_zero) > 100
plt.imshow(img_zero, cmap="gray")

print(np.argmax(model_cnn.predict(img_zero.reshape((1,28,28,1)))))

In [0]:
img_um = cv.imread("um.jpg", cv.IMREAD_GRAYSCALE)
img_um = (255-img_um) > 100
plt.imshow(img_um, cmap="gray")

print(np.argmax(model_cnn.predict(img_um.reshape((1,28,28,1)))))